<a href="https://colab.research.google.com/github/vinayshanbhag/cloudvisionproductsearch/blob/main/Google_Cloud_Vision_Create_a_product_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Google Vision API Product Search allows retailers to create products, each containing reference images that visually describe the product from a set of viewpoints. Retailers can then add these products to product sets. 

This Notebook demonstrates how to create and manage product sets for use with the Google Cloud Vision Product Search API.

After a product set is created users can query the product set with their own images, Vision API Product Search applies machine learning to compare the product in the user's query image with the images in the retailer's product set, and then returns a ranked list of visually and semantically similar results. This is covered in a separate Notebook [<sup>[Google Cloud Vision Product Search.ipynb]</sup>](https://github.com/vinayshanbhag/cloudvisionproductsearch/blob/main/Google_Cloud_Vision_Product_Search.ipynb)




# 1. Before you begin
1. Create a project and service account [<sup>[1]</sup>](https://cloud.google.com/vision/product-search/docs/before-you-begin)
2. Create a Google Cloud Storage bucket [<sup>[2]</sup>](https://cloud.google.com/storage/docs/creating-buckets)
3. Grant service account view access to this bucket [<sup>[3]</sup>](https://cloud.google.com/storage/docs/access-control)
4. Upload images files to the bucket

# 2. Bulk Import

Product sets can be imported in bulk using a CSV file with following values (no header) [<sup>[4]</sup>](https://cloud.google.com/vision/product-search/docs/csv-format)
* image-uri: Google cloud storage uri
* image-id: Unique identifier for image
* product-set-id: A unique identifier for the product set to import the images into
* product-id: A product-id can be associated with multiple reference images.
* product-category: One of {homegoods-v2 | apparel-v2 | toys-v2 | packagedgoods-v1 | general-v1}
* product-display-name: The product name
* labels: labels if any
* bounding-poly: Bounding polygon coordinates of the product in image

A sample CSV file...

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/vinayshanbhag/images/main/products.csv',header=None, names=['image-uri','image-id','product-set-id','product-id','product-category','product-display-name','labels','bounding-poly']).fillna("")
df.sample(5)

,image-uri,image-id,product-set-id,product-id,product-category,product-display-name,labels,bounding-poly
47,gs://cvproducts/galaxy_s20_ultra_5g_1.jpg,galaxy_s20_ultra_5g_1,smartdevices,galaxy_s20_ultra_5g,general-v1,Galaxy S20 Ultra 5G,,
37,gs://cvproducts/nest_doorbell_battery_1.jpg,nest_doorbell_battery_1,smartdevices,nest_doorbell_battery,general-v1,Nest Doorbell Battery,,
68,gs://cvproducts/galaxy_s21_5g_1.jpg,galaxy_s21_5g_1,smartdevices,galaxy_s21_5g,general-v1,Galaxy S21 5G,,
3,gs://cvproducts/galaxy_s20_1.jpg,galaxy_s20_1,smartdevices,galaxy_s20,general-v1,Galaxy S20,,
62,gs://cvproducts/apple_watch_series_3_2.jpg,apple_watch_series_3_2,smartdevices,apple_watch_series_3,general-v1,Apple Watch Series 3,,


This CSV import file must be uploaded to a Google Cloud Storage bucket.

# 3. Installation
Install Google Cloud Vision packages required to create and manage product sets.

`pip3 install -U pip google-cloud-vision`

Restart kernel, if prompted.

In [12]:
#install google cloud vision
!pip3 install -U pip google-cloud-vision

     |████████████████████████████████| 1.7 MB 35.0 MB/s 
     |████████████████████████████████| 370 kB 59.2 MB/s 
     |████████████████████████████████| 45 kB 3.7 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 44.3 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires goog

# 4. Configure credentials

Upload application credentials (key.json) file and set `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to that file.

In [11]:
import google.cloud.vision
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/key.json"

# 5. Create a product set

To create a new product set, we need-
* project_id: Id of the project (created in 1.1)
* location: A compute region name.e.g.'us-west1' (created in 1.1)
* product_set_id: Id of the product set. 
* product_set_display_name: Display name of the product set

In [2]:
from google.cloud import vision

def create_product_set(
        project_id, location, product_set_id, product_set_display_name):
    """Create a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_set_display_name: Display name of the product set.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Create a product set with the product set specification in the region.
    product_set = vision.ProductSet(
            display_name=product_set_display_name)

    # The response is the product set with `name` populated.
    response = client.create_product_set(
        parent=location_path,
        product_set=product_set,
        product_set_id=product_set_id)

    # Display the product set information.
    print('Product set name: {}'.format(response.name))

In [3]:
#create_product_set('cvproductsearch','us-west1','smartdevices','Smart Devices')

# 6. Import product sets from CSV

* Bulk import CSV file must be formatted as described in 2.
* The file must be in a Google cloud storage bucket, and that storage bucket must be accessible to the service account credentials in key.json
* All image files referenced in the CSV must be in a Google cloud storage bucket, and that storage bucket must be accessible to the service account credentials in `key.json`
* To import products from the CSV, we need project_id, location and Google cloud storage uri for the CSV import file

After product sets are imported, it takes up to 30 minutes for these products to be indexed. These products will not show up in user queries, until they are indexed. 

In [4]:
def import_product_sets(project_id, location, gcs_uri):
    """Import images of different products in the product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        gcs_uri: Google Cloud Storage URI.
            Target files must be in Product Search CSV format.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Set the input configuration along with Google Cloud Storage URI
    gcs_source = vision.ImportProductSetsGcsSource(
        csv_file_uri=gcs_uri)
    input_config = vision.ImportProductSetsInputConfig(
        gcs_source=gcs_source)

    # Import the product sets from the input URI.
    response = client.import_product_sets(
        parent=location_path, input_config=input_config)

    print('Processing operation name: {}'.format(response.operation.name))
    # synchronous check of operation status
    result = response.result()
    print('Processing done.')

    for i, status in enumerate(result.statuses):
        print('Status of processing line {} of the csv: {}'.format(
            i, status))
        # Check the status of reference image
        # `0` is the code for OK in google.rpc.Code.
        if status.code == 0:
            reference_image = result.reference_images[i]
            print(reference_image)
        else:
            print('Status code not OK: {}'.format(status.message))

In [5]:
#import_product_sets('cvproductsearch', 'us-west1', 'gs://cvproducts/products.csv')

# 7. List products in a product set

In [6]:
def list_products_in_product_set(
        project_id, location, product_set_id):
    """List all products in a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # List all the products available in the product set.
    products = client.list_products_in_product_set(name=product_set_path)

    # Display the product information.
    for product in products:
        print('Product name: {}'.format(product.name))
        print('Product id: {}'.format(product.name.split('/')[-1]))
        print('Product display name: {}'.format(product.display_name))
        print('Product description: {}'.format(product.description))
        print('Product category: {}'.format(product.product_category))
        print('Product labels: {}'.format(product.product_labels))

In [8]:
list_products_in_product_set(project_id='cvproductsearch', location='us-west1', product_set_id='smartdevices')

Product name: projects/cvproductsearch/locations/us-west1/products/airpods_2
Product id: airpods_2
Product display name: Airpods 2
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/cvproductsearch/locations/us-west1/products/apple_watch_series_3
Product id: apple_watch_series_3
Product display name: Apple Watch Series 3
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/cvproductsearch/locations/us-west1/products/apple_watch_series_4
Product id: apple_watch_series_4
Product display name: Apple Watch Series 4
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/cvproductsearch/locations/us-west1/products/apple_watch_series_5
Product id: apple_watch_series_5
Product display name: Apple Watch Series 5
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/cvproductsearch/locations/us-west1/products/apple_watch_series_6
Product i

# 8. Delete a product set

In [9]:
def delete_product_set(project_id, location, product_set_id):
    """Delete a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # Delete the product set.
    client.delete_product_set(name=product_set_path)
    print('Product set deleted.')

In [10]:
# delete_product_set("cvproductsearch","us-west1","smartdevices")